In [101]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score

## Load data

### Initialize path constant

In [102]:
DATA_PATH = '../data'
RAW_DATA_PATH = DATA_PATH+'/raw'
PROCESSED_DATA_PATH = DATA_PATH+'/processed'

### Read processed data from CSV

In [103]:
data = pd.read_csv('{}/processed_data.csv'.format(PROCESSED_DATA_PATH))
X = data.drop(['heart_disease_diagnosis'], axis=1)
y = data['heart_disease_diagnosis']

## Cross Validation

#### Train-test split

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X.values,y.values, test_size=0.33)

## Modelling

### Random Forest Classifier

In [105]:
model = RandomForestClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_pred=y_pred, y_true=y_test)
precision = precision_score(y_pred=y_pred, y_true=y_test, average='macro')
recall = recall_score(y_pred=y_pred, y_true=y_test, average='macro')

print("Accuracy = {}".format(accuracy))
print("Precision = {}".format(precision))
print("Recall = {}".format(recall))

Accuracy = 0.5581395348837209
Precision = 0.515692007797271
Recall = 0.3454718964660136


/home/adylan/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
